In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

In [3]:
# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query_1014 = '''
    SELECT 
    (COALESCE(a.score_2, 0) + COALESCE(a.score_5b, 0) + COALESCE(a.score_6a, 0) + COALESCE(a.score_6c, 0) + COALESCE(a.score_61b, 0) + COALESCE(a.score_17a, 0)) AS schooling_score_total,
    p.patient_code AS dreams_code,
    dga.nbr_presence,
    a.*
FROM
    (SELECT 
        CASE
                WHEN d.paran_ou_vivan = '0' THEN 0
                WHEN d.paran_ou_vivan = '2' THEN 2
                WHEN d.paran_ou_vivan = '4' THEN 5
            END AS score_2,
            CASE
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '1' THEN 1
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '2' THEN 2
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '3' THEN 5
            END AS score_5b,
            CASE
                WHEN d.a_ske_ou_lekl = '0' THEN 2
                WHEN d.a_ske_ou_lekl = '6' THEN 0
            END AS score_6a,
            d.c6_ske_ou_te_oblije_double_deja = 0 AS score_6c,
            CASE
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '00' THEN 0
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '0' THEN 1
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '5' THEN 5
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '000' THEN - 50
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '05' THEN 5
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '001' THEN 2
                WHEN d.c6b_kils_ki_peye_lekl_ou_10_14 = '8' THEN 10
            END AS score_61b,
            CASE
                WHEN d.a17 = '0' THEN 0
                WHEN d.a17 = '4' THEN 10
                WHEN d.a17 = '04' THEN 10
                WHEN d.a17 = '004' THEN 10
            END AS score_17a,
            d.*
    FROM
        caris_db.dreams_surveys_data d) a
        LEFT JOIN
    dream_member dm ON dm.case_id = a.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    (SELECT 
        COUNT(DISTINCT dgs.topic) AS nbr_presence, id_patient
    FROM
        dream_group_attendance dga1
    LEFT JOIN dream_group_session dgs ON dga1.id_group_session = dgs.id
    WHERE
        dga1.value = 'P'
    GROUP BY dga1.id_patient) dga ON dga.id_patient = dm.id_patient
WHERE
    a.total > 14
ORDER BY (a.score_2 + a.score_5b + a.score_6a + a.score_6c + a.score_61b + a.score_17a) DESC
'''

dreams_schooling_1014 = pd.read_sql_query(query_1014,engine,parse_dates=True)

In [4]:
# get the engine to connect and fetch
query_1518 = '''
    SELECT 
    (COALESCE(a.score_2, 0) + COALESCE(a.score_5b, 0) + COALESCE(a.score_6a, 0) + COALESCE(a.score_6c, 0) + COALESCE(a.score_61b, 0) + COALESCE(a.score_17a, 0)) AS schooling_score_total,
    p.patient_code AS dreams_code,
    dga.nbr_presence,
    a.*
FROM
    (SELECT 
        CASE
                WHEN d.paran_ou_vivan = '0' THEN 0
                WHEN d.paran_ou_vivan = '2' THEN 2
                WHEN d.paran_ou_vivan = '4' THEN 3
            END AS score_2,
            CASE
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '1' THEN 1
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '2' THEN 2
                WHEN d.b5_konbyen_fwa_ou_al_dmi_tou_grangou_pandan_kat_dnye_semn_yo = '3' THEN 5
            END AS score_5b,
            CASE
                WHEN d.tranche_age_6a_15_19 = '0' THEN 0
                WHEN d.tranche_age_6a_15_19 = '3' THEN 2
            END AS score_6a,
            d.c6_ske_ou_te_oblije_double_deja  AS score_6c,
            CASE
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '0' THEN 0
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '00' THEN 1
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '3' THEN 3
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '000' THEN - 50
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '03' THEN 5
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '01' THEN 2
                WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '6' THEN 10
            END AS score_61b,
			CASE
                WHEN d.a17 = '0' THEN 0
                WHEN d.a17 = '4' THEN 10
                WHEN d.a17 = '04' THEN 10
                WHEN d.a17 = '004' THEN 10
            END AS score_17a,
            d.*
    FROM
        caris_db.dreams_surveys_data d) a
        LEFT JOIN
    dream_member dm ON dm.case_id = a.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    (SELECT 
        COUNT(DISTINCT dgs.topic) AS nbr_presence, id_patient
    FROM
        dream_group_attendance dga1
    LEFT JOIN dream_group_session dgs ON dga1.id_group_session = dgs.id
    WHERE
        dga1.value = 'P'
    GROUP BY dga1.id_patient) dga ON dga.id_patient = dm.id_patient
WHERE
    a.total > 14
ORDER BY (a.score_2 + a.score_5b + a.score_6a + a.score_6c + a.score_61b + a.score_17a) DESC
'''

dreams_schooling_1518 = pd.read_sql_query(query_1518,engine,parse_dates=True)

In [5]:
# close the pool of connection
engine.dispose()

## Disponible

In [6]:
dreams_schooling_1014.dtypes

schooling_score_total             int64
dreams_code                      object
nbr_presence                    float64
score_2                           int64
score_5b                        float64
                              ...      
timeEnd                  datetime64[ns]
timeStart                datetime64[ns]
agent_username                   object
created_ad               datetime64[ns]
organisation                     object
Length: 210, dtype: object

In [7]:
dreams_schooling_1014.shape

(40743, 210)

In [8]:
dreams_schooling_1518.dtypes

schooling_score_total           float64
dreams_code                      object
nbr_presence                    float64
score_2                           int64
score_5b                        float64
                              ...      
timeEnd                  datetime64[ns]
timeStart                datetime64[ns]
agent_username                   object
created_ad               datetime64[ns]
organisation                     object
Length: 210, dtype: object

In [9]:
dreams_schooling_1518.shape

(40743, 210)

## Excellence

In [10]:
dreams_schooling_1014.to_excel('dreams_schooling_1014.xlsx',index=False,na_rep="")
dreams_schooling_1518.to_excel('dreams_schooling_1518.xlsx',index=False,na_rep="")

In [11]:
dreams_schooling_1014.id.count()

40743

In [12]:

dreams_schooling_1518.id.count()

40743